In [2]:
import os
import pdfplumber
import pandas as pd
import logging
from pathlib import Path


In [6]:
import os
import pdfplumber

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ""
        for page in pdf.pages:
            all_text += page.extract_text() + "\n"
    return all_text

def extract_text_from_folder(folder_path, output_folder='txt2'):
    """Extracts text from all PDFs in a folder and saves them as .txt files."""
    # If output folder is None, use default name
    if output_folder is None:
        output_folder = os.path.join(folder_path, "extracted_texts")

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the folder
    for root, dirs, files in os.walk(folder_path):
        print('===================================')
        print(f'Processing folder: {root}')
        print(f'Files found: {files}')
        print('===================================')
        
        for file in files:
            if file.endswith(".pdf"):
                file_path = os.path.join(root, file)
                try:
                    # Extract text from PDF
                    extracted_text = extract_text_from_pdf(file_path)

                    # Generate output filename (use the folder name to avoid issues with filename lengths)
                    output_filename = os.path.splitext(file)[0] + ".txt"
                    output_path = os.path.join(output_folder, output_filename)

                    # Write extracted text to file
                    with open(output_path, "w", encoding="utf-8") as f:
                        f.write(extracted_text)

                    print(f"Extracted text from {file_path}")

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Example usage
folder_path = "aaa"  # Example folder path containing PDFs
extract_text_from_folder(folder_path)


Processing folder: aaa
Files found: []
Processing folder: aaa\2023_Q4
Files found: ['24CS_86500201.pdf', '2S_86215801.pdf', '3BBIF_85806601.pdf', '3K-BAT_85852201.pdf', 'A5_86520501.pdf', 'AAI_86094601.pdf', 'AAV_86270501.pdf', 'ABM_86368601.pdf', 'ACC_86663601.pdf', 'ACE_86706701.pdf', 'ACG_86119901.pdf', 'ADB_86150001.pdf', 'ADD_86212601.pdf', 'ADVANC_85814801.pdf', 'ADVANC_86685701.pdf', 'ADVICE_85728701.pdf', 'ADVICE_85729401.pdf', 'ADVICE_86439501.pdf', 'AEONTS_85538501.pdf', 'AE_86469401.pdf', 'AFC_85874901.pdf', 'AF_86136601.pdf', 'AGE_86062001.pdf', 'AGE_86072501.pdf', 'AHC_86247501.pdf', 'AH_86765601.pdf', 'AIE_86078001.pdf', 'AIMCG_86617601.pdf', 'AIMIRT_86618001.pdf', 'AIRA_86696801.pdf', 'AIT_85968401.pdf', 'AI_86079501.pdf', 'AJA_86370001.pdf', 'AJ_86511801.pdf', 'AKP_86164601.pdf', 'AKR_86607101.pdf', 'ALLA_86252401.pdf', 'ALLY_85888301.pdf', 'ALL_85500201.pdf', 'ALL_85507301.pdf', 'ALPHAX_86607201.pdf', 'ALT_86219401.pdf', 'ALUCON_86350401.pdf', 'AMANAH_86031801.pdf', 'A

KeyboardInterrupt: 

In [9]:
def create_csv_from_txt(folder_path, output_csv_path):
    data = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):  # ตรวจสอบว่าเป็นไฟล์ .txt
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read().strip()  # อ่านเนื้อหาไฟล์และตัดช่องว่างที่ไม่จำเป็น
                data.append({'Filename': filename, 'content': content})
    
    # สร้าง DataFrame
    df = pd.DataFrame(data)
    
    # บันทึกเป็น CSV โดยใช้ escapechar
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig', escapechar='\\')  # เพิ่ม escapechar
    print(f"บันทึกไฟล์ CSV เรียบร้อย: {output_csv_path}")

# ตัวอย่างการใช้งาน
folder_path = "txt"  # เปลี่ยนเป็น path ของโฟลเดอร์ที่มีไฟล์ .txt
output_csv_path = "txt_csv.csv"  # เปลี่ยนเป็น path ของไฟล์ .csv ที่ต้องการบันทึก
create_csv_from_txt(folder_path, output_csv_path)


บันทึกไฟล์ CSV เรียบร้อย: txt_csv.csv


In [18]:
import pandas as pd
import re

def check_section_exists(text, keywords):
    text_lower = str(text).lower()
    return 'Y' if any(keyword.lower() in text_lower for keyword in keywords) else 'N'

def process_csv(input_file, output_file):
    df = pd.read_csv(input_file)
    sections = {
        'ภาพรวม': ['สรุป', 'การดำเนินงาน', 'ผลการดำเนินงาน', 'สถานการณ์'],
        'สภาวะตลาด': ['ราคาขาย', 'เหล็กโลก', 'การปรับตัว', 'ตลาด'],
        'สรุปการผลการดำเนินงาน': ['กำไรสุทธิ', 'รายได้', 'อัตรากำไร', 'รายงานผล', 'การเปลี่ยนแปลง'],
        'เหตุการณ์สำคัญ': ['การเปลี่ยนแปลง', 'การเพิ่มขึ้น', 'การปรับตัว'],
        'ปัจจัยต่อการเติบโต': ['ราคาขายเฉลี่ย', 'การบริหาร', 'สินค้าคงคลัง', 'การเพิ่มขึ้นของรายได้'],
        'เปิดเผยรายได้': ['รายได้จากการขาย', 'การเพิ่มขึ้นของรายได้', 'ราคาขาย'],
        'เปิดเผยกำไรที่เปลี่ยนไป': ['กำไรสุทธิ', 'การเพิ่มขึ้น', 'การเปลี่ยนแปลงกำไร'],
        'คำอธิบายราย': ['การอธิบาย', 'สาเหตุ', 'ผลลัพธ์']
    }
    
    # Add new columns with Y/N flags
    for section, keywords in sections.items():
        df[section] = df['content'].apply(lambda x: check_section_exists(x, keywords))
    
    # Save to new CSV
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"Processing complete. Output saved to {output_file}")

# Example usage
process_csv('txt_csv.csv', 'NLP.csv')

Processing complete. Output saved to NLP.csv


In [19]:
df = pd.read_csv('NLP.csv')

In [20]:
df.head()

,Filename,content,ภาพรวม,สภาวะตลาด,สรุปการผลการดำเนินงาน,เหตุการณ์สำคัญ,ปัจจัยต่อการเติบโต,เปิดเผยรายได้,เปิดเผยกำไรที่เปลี่ยนไป,คำอธิบายราย
0,24CS_86500201.txt,TWENTY-FOUR CON & SUPPLY PUBLIC COMPANY LIMITE...,N,Y,Y,N,N,Y,N,Y
1,2S_86215801.txt,ท ่ี2S_BoD002/2567\nวนั ท ่ี22 กมุ ภาพนั ธ ์25...,N,Y,N,N,N,Y,N,Y
2,3BBIF_85806601.txt,กองทนุ รวมโครงสรา้ งพนื้ ฐานบรอดแบนดอ์ นิ เทอร...,N,N,N,N,N,N,N,N
3,3K-BAT_85852201.txt,ท(cid:201) ีCS-2401002 9 กมุ ภาพนั ธ ์2567\nเร...,N,Y,N,N,Y,Y,N,Y
4,A5_86520501.txt,บริษทั แอสเซท ไฟว ์กรุ๊ป จำ กดั (มหำชน) เลขที่...,N,Y,N,N,Y,N,N,Y


In [23]:
df.drop(["content"], axis = 1, inplace=True)

In [25]:
def calculate_report_quality(row):
    count_y = sum(1 for value in row[1:] if value == 'Y')
    if count_y >= 4:
        return 'A'
    elif count_y < 2:
        return 'C'
    else:
        return 'B'

df['Filename'] = df['Filename'].str.replace('.txt', '', regex=False)

df['Report_quality'] = df.apply(calculate_report_quality, axis=1)

df = df[['Filename', 'Report_quality'] + [col for col in df.columns if col not in ['Filename', 'Report_quality']]]
df = df.sort_values(by='Filename')

In [26]:
df.head()

,Filename,Report_quality,ภาพรวม,สภาวะตลาด,สรุปการผลการดำเนินงาน,เหตุการณ์สำคัญ,ปัจจัยต่อการเติบโต,เปิดเผยรายได้,เปิดเผยกำไรที่เปลี่ยนไป,คำอธิบายราย
0,24CS_86500201,A,N,Y,Y,N,N,Y,N,Y
1,2S_86215801,B,N,Y,N,N,N,Y,N,Y
2,3BBIF_85806601,C,N,N,N,N,N,N,N,N
3,3K-BAT_85852201,A,N,Y,N,N,Y,Y,N,Y
4,A5_86520501,B,N,Y,N,N,Y,N,N,Y


In [27]:
try:
    df.to_csv('classified_with_quality.csv', index=False, encoding='utf-8-sig')
    print("บันทึกไฟล์สำเร็จ")
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการบันทึกไฟล์: {str(e)}")

บันทึกไฟล์สำเร็จ
